<a href="https://colab.research.google.com/github/Modnarudom/modProject/blob/main/churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Churn model


In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import seaborn as sns
#-----------

###import data

In [ ]:
!unzip /content/data.zip

Archive:  /content/data.zip
  inflating: WA_Fn-UseC_-Telco-Customer-Churn.csv  


In [ ]:
data = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
data.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [ ]:
print("Dataset size")
print("Rows {} Columns {}".format(data.shape[0], data.shape[1]))

Dataset size
Rows 7043 Columns 20


In [ ]:
print("Columns and data types")
pd.DataFrame(data.dtypes).rename(columns = {0:'dtype'})

Columns and data types


,dtype
gender,object
SeniorCitizen,int64
Partner,object
Dependents,object
tenure,int64
PhoneService,object
MultipleLines,object
InternetService,object
OnlineSecurity,object
OnlineBackup,object


In [ ]:
data.describe(include= "O").T

,count,unique,top,freq
customerID,7043,7043,7590-VHVEG,1
gender,7043,2,Male,3555
Partner,7043,2,No,3641
Dependents,7043,2,No,4933
PhoneService,7043,2,Yes,6361
MultipleLines,7043,3,No,3390
InternetService,7043,3,Fiber optic,3096
OnlineSecurity,7043,3,No,3498
OnlineBackup,7043,3,No,3088
DeviceProtection,7043,3,No,3095


###clean data

In [ ]:
data.drop("customerID", axis =1 , inplace = True)
data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
def check_missing_values(data):
  missing_value = data.isnull().sum().sort_values(ascending = False)
  percent_missing = (data.isnull().sum()/data.shape[0]).sort_values(ascending = False)
  missing = pd.concat([missing_value, percent_missing], axis = 1, keys =['missing_value', 'percent_missing'])
  return missing
check_missing_values(data)

,missing_value,percent_missing
gender,0,0.0
SeniorCitizen,0,0.0
TotalCharges,0,0.0
MonthlyCharges,0,0.0
PaymentMethod,0,0.0
PaperlessBilling,0,0.0
Contract,0,0.0
StreamingMovies,0,0.0
StreamingTV,0,0.0
TechSupport,0,0.0


In [ ]:
data["TotalCharges"] = data["TotalCharges"].str.replace(" ", "").replace("", np.nan).astype(float)
data.describe(include = 'O').T

,count,unique,top,freq
gender,7043,2,Male,3555
Partner,7043,2,No,3641
Dependents,7043,2,No,4933
PhoneService,7043,2,Yes,6361
MultipleLines,7043,3,No,3390
InternetService,7043,3,Fiber optic,3096
OnlineSecurity,7043,3,No,3498
OnlineBackup,7043,3,No,3088
DeviceProtection,7043,3,No,3095
TechSupport,7043,3,No,3473


In [ ]:
check_missing_values(data)

,missing_value,percent_missing
TotalCharges,11,0.001562
gender,0,0.000000
SeniorCitizen,0,0.000000
MonthlyCharges,0,0.000000
PaymentMethod,0,0.000000
PaperlessBilling,0,0.000000
Contract,0,0.000000
StreamingMovies,0,0.000000
StreamingTV,0,0.000000
TechSupport,0,0.000000


In [ ]:
columns = data.columns
numeric = {}
category = {}
for col in columns:
  if data[col].dtypes != object:
    numeric[col] = len(data[col].unique())
  else:
    category[col] = len(data[col].unique())




In [ ]:
print(f"--------------numeric feild----------------")
for key,value in numeric.items():
  print(f"unique number of {key} = {value}\n")

print(f"--------------category feild----------------")
for key,value in category.items():
  print(f"unique number of {key} = {value} \n")


--------------numeric feild----------------
unique number of SeniorCitizen = 2

unique number of tenure = 73

unique number of MonthlyCharges = 1585

unique number of TotalCharges = 6531

--------------category feild----------------
unique number of gender = 2 

unique number of Partner = 2 

unique number of Dependents = 2 

unique number of PhoneService = 2 

unique number of MultipleLines = 3 

unique number of InternetService = 3 

unique number of OnlineSecurity = 3 

unique number of OnlineBackup = 3 

unique number of DeviceProtection = 3 

unique number of TechSupport = 3 

unique number of StreamingTV = 3 

unique number of StreamingMovies = 3 

unique number of Contract = 3 

unique number of PaperlessBilling = 2 

unique number of PaymentMethod = 4 

unique number of Churn = 2 

